In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!',
 'explanation': 'This joke plays on the double meaning of the word "crusty." In one sense, it can mean having a hard, dry surface like the crust of a pizza. In another sense, it can mean feeling irritable or grumpy. By saying that the pizza went to the doctor because it was feeling "crusty," the joke is suggesting that the pizza was not only physically crusty but also in a bad mood. This creates a silly and playful image of a pizza seeking medical help for its physical and emotional state.'}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!', 'explanation': 'This joke plays on the double meaning of the word "crusty." In one sense, it can mean having a hard, dry surface like the crust of a pizza. In another sense, it can mean feeling irritable or grumpy. By saying that the pizza went to the doctor because it was feeling "crusty," the joke is suggesting that the pizza was not only physically crusty but also in a bad mood. This creates a silly and playful image of a pizza seeking medical help for its physical and emotional state.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-570e-648d-8002-f85918f1b19c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-05T15:09:21.230964+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-4937-68b8-8001-f99da3e96f3b'}}, tasks=(), in

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!', 'explanation': 'This joke plays on the double meaning of the word "crusty." In one sense, it can mean having a hard, dry surface like the crust of a pizza. In another sense, it can mean feeling irritable or grumpy. By saying that the pizza went to the doctor because it was feeling "crusty," the joke is suggesting that the pizza was not only physically crusty but also in a bad mood. This creates a silly and playful image of a pizza seeking medical help for its physical and emotional state.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-570e-648d-8002-f85918f1b19c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-05T15:09:21.230964+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-4937-68b8-8001-f99da3e96f3b'}}, tasks=(), i

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)


{'topic': 'pasta',
 'joke': 'Why did the pasta blush? Because it saw the spaghetti sauce!',
 'explanation': 'This joke is a play on words, using a pun to create humor. In this case, the joke refers to pasta blushing, which is obviously not something that can happen in reality. The punchline "Because it saw the spaghetti sauce!" is a clever twist on the idea of pasta blushing - suggesting that the pasta blushed because it was embarrassed or shy after seeing the spaghetti sauce, as if it were in a romantic situation. The humor comes from the unexpected connection between the pasta and sauce, and the absurdity of the idea that pasta could actually blush.'}

In [11]:
workflow.get_state(config2)


StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta blush? Because it saw the spaghetti sauce!', 'explanation': 'This joke is a play on words, using a pun to create humor. In this case, the joke refers to pasta blushing, which is obviously not something that can happen in reality. The punchline "Because it saw the spaghetti sauce!" is a clever twist on the idea of pasta blushing - suggesting that the pasta blushed because it was embarrassed or shy after seeing the spaghetti sauce, as if it were in a romantic situation. The humor comes from the unexpected connection between the pasta and sauce, and the absurdity of the idea that pasta could actually blush.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd7-060d-6922-8002-c964c0da0637'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-05T15:10:33.267843+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_i

In [12]:
list(workflow.get_state_history(config2))


[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta blush? Because it saw the spaghetti sauce!', 'explanation': 'This joke is a play on words, using a pun to create humor. In this case, the joke refers to pasta blushing, which is obviously not something that can happen in reality. The punchline "Because it saw the spaghetti sauce!" is a clever twist on the idea of pasta blushing - suggesting that the pasta blushed because it was embarrassed or shy after seeing the spaghetti sauce, as if it were in a romantic situation. The humor comes from the unexpected connection between the pasta and sauce, and the absurdity of the idea that pasta could actually blush.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd7-060d-6922-8002-c964c0da0637'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-05T15:10:33.267843+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_

Time Travel

In [16]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0a1fd4-3c07-673c-8000-fbdc4334050e"}})


StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0a1fd4-3c07-673c-8000-fbdc4334050e'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-10-05T15:09:18.397010+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-3bd9-60fd-bfff-395af157350e'}}, tasks=(PregelTask(id='a38e26b0-6272-5bdd-48b3-12bb0ecd3acb', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the doctor? Because it was feeling a little crusty!'}),), interrupts=())

In [17]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0a1fd4-3c07-673c-8000-fbdc4334050e"}})


{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': 'This joke plays on the idea of a pizza having "toppings" like a person might have "issues" or "problems." In this case, the pizza is portrayed as needing therapy because it has too many toppings, causing it stress or pressure. The juxtaposition of a pizza seeking therapy adds humor to the situation.'}

In [18]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea of a pizza having "toppings" like a person might have "issues" or "problems." In this case, the pizza is portrayed as needing therapy because it has too many toppings, causing it stress or pressure. The juxtaposition of a pizza seeking therapy adds humor to the situation.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a2010-0347-60ce-8002-9a98ee68c79f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-05T15:36:03.058913+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a200f-f424-667f-8001-4a887df4d8b4'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn

In [20]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0a1fd4-3c07-673c-8000-fbdc4334050e", "checkpoint_ns": ""}}, {'topic':'samosa'})


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0a2016-6c96-6552-8001-f506832654d3'}}

In [21]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a2016-6c96-6552-8001-f506832654d3'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-10-05T15:38:55.162709+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a1fd4-3c07-673c-8000-fbdc4334050e'}}, tasks=(PregelTask(id='e4aaacb3-325d-12bc-01ed-be98c9cfce39', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a2010-cea2-63f4-8001-42b7d1dc41e8'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-10-05T15:36:24.382360+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id

In [22]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0a2016-6c96-6552-8001-f506832654d3"}})


{'topic': 'samosa',
 'joke': 'Why did the samosa go to the party? \nBecause it wanted to be the life of the snack table!',
 'explanation': 'This joke plays on the idea of the samosa, a popular Indian snack, wanting to be the center of attention at a party. By saying it wanted to be the life of the snack table, the joke suggests that the samosa wanted to be the most popular and enjoyed snack at the party, bringing humor to the idea of a snack having a social life and personality.'}

In [23]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the party? \nBecause it wanted to be the life of the snack table!', 'explanation': 'This joke plays on the idea of the samosa, a popular Indian snack, wanting to be the center of attention at a party. By saying it wanted to be the life of the snack table, the joke suggests that the samosa wanted to be the most popular and enjoyed snack at the party, bringing humor to the idea of a snack having a social life and personality.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a2018-5544-69fc-8003-5412a5350f0e'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-10-05T15:39:46.404607+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a2018-48a8-6eb0-8002-299f10274dda'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the party? \nBecause it wan